The calculations of loss during the training of networks can use weighting to reinforce the learning of specific variables and features in data. In order to reinforce physically reasonable predictions, you can impose more strict penalties for predictions in physically invalid predictions. For example, a prediction of Power Conversion Efficiency (PCE) above the Schockley-Quassier predicted maximum PCE could increase exponentially, rather than linearly.

The classes developed below are wrappers for PyTorch tensors loss functions, which additionally modify these classes using theoretically and emprically derived boundaries for network loss calculations.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class OPV_MSE(nn.Module):
    """
    This class contains loss functions that use a mean-squared-error loss for reasonable predictions.
    They inherit from torch.nn.Module just like the custom model. For physically unreasonable conditions,
    prediction loss is more severely calculated. What qualifies as reasonable is based on empirically
    gathered datasets and literature reported boundaries of performance in P3HT:PCBM OPV devices.
    """
    
    def __init__(self):
        super(PhysicsInspiredLoss, self).__init()

    def PCE_Loss(self, predictions, targets):
        """
        For the following Power Conversion Efficiency predictions that are improbable, the loss is penalized:
        - PCE < 0%
        - PCE > 6%
        """
            for x, y in predictions, labels:

            if x < 0:
                #quadratic MSE
                loss = F.mse_loss(x,y)
                loss = torch squaring function(loss)


            elif x > 0.06:
                #quadratic MSE

            else:
                loss = F.mse_loss()

        return loss

    def Voc_Loss(predictions, targets):
        """
         For the following open-circuitt voltage predictions that are improbable, the loss is penalized:
        - Voc < 0
        - Voc > 1.0
        """
        if predictions < 0:
            #quadratic MSE

        elif predictions > 1.0:
            #quadratic MSE

        else:
            loss = nn.MSELoss(reduction = 'mean')

        return loss

    def Jsc_Loss(predictions, targets):
        """
         For the following short-circuit current predictions that are improbable, the loss is penalized:
        - Jsc < 0
        - Jsc > 10
        """
        if predictions < 0:
            #quadratic MSE

        elif predictions > 10:
            #quadratic MSE

        else:
            loss = nn.MSELoss(reduction = 'mean')

        return loss

    def FF_Loss(predictions, targets):
        """
         For the following Fill Factor predictions that are improbable, the loss is penalized:
        - FF < 10
        - FF > 85
        """
        if predictions < 10:
            #quadratic MSE

        elif predictions > 85:
            #quadratic MSE

        else:
            loss = nn.MSELoss(reduction = 'mean')

        return loss